1- Verification Coherence entre le THI initial du tableau update et le THI initial dans les outputs

Nous soupconnons une erreur d'inatention mais nous allons tout de meme verifier à l'aide de cet algorithme qui va verifier la correlation entre ces deux features qui devraient, par notre logiques avoir les memes resultats pour chaque patient.

- Importation des librairies

In [140]:
import pandas as pd
from math import *
import numpy as np
from colorama import Back, Fore, Style, deinit, init

- Ouverture du fichier de données

In [141]:
fichier_excel = "KOPACOU_Sybilone/wetransfer_usecase-orl_2024-05-27_0918/UseCase ORL/orl_acouphene/data/ETUDE COOPACOU XLDEF update copy.xlsx"
data = pd.read_excel(fichier_excel, dtype=str)

Tout d'abord, regardons le nombre de données mises à jour fournis: 

In [142]:
def nb_patient_nouvelles_data(index2):
    outputs = data.iloc[index2].values[1:]
    i = 0
    while pd.notna(outputs[i]):
        i += 1
    return i

In [143]:
taille = nb_patient_nouvelles_data(335)
print(taille)

100


On aura donc des un jeux de 100 données pour cette étude

Cependant ces données ne sont pas toujours cohérentes. Les données de bases ne sont pas toujours égales aux données mises à jour.

On va donc devoir dans un premier temps verifier la cohérence entre les données.

I- Cohérence des données :

Voici une premiere fonction qui va verifier la cohérence entre une ligne et une autre du tableau de données et fournir le pourcentage de correspondance entre les données de ces deux lignes :

Avant propos : Décodage des données

In [144]:
mention_refus_appareillages=["REFUSE","NON","Non"]

In [145]:
mention_vide = ["nan","0"]

In [146]:
mention_acceptation_appareillage =["APP"]

In [147]:
def appartenance_au_moins_un(liste,str):
    for index in liste :
        if index in str :
            return True
    return False

In [148]:
def creation_tab_colonne_personnalisé(liste):
    tab=[]
    for i in liste:
        tab.append(data.columns[i])
    return tab

In [149]:
def correspondance_simple(index1, index2, subject, i):

    correspondance=[]
    non_correspondance=[]
    
    ligne_1 = data.iloc[index1].values[1:]
    ligne_2 = data.iloc[index2].values[1:]
    ligne_1 = ligne_1[:i]
    
    for index in range(i):
        if ligne_1[index] != ligne_2[index]:
            if appartenance_au_moins_un(mention_acceptation_appareillage,str(ligne_1[index])):
                if "APP"[0] == ligne_2[index]:
                    correspondance.append(index+1)
            
            elif appartenance_au_moins_un(mention_refus_appareillages,str(ligne_1[index])):
                ligne_1[index] = np.nan
                if pd.isna(ligne_1[index]) and pd.isna(ligne_2[index]):
                    correspondance.append(index+1)
                elif appartenance_au_moins_un(mention_vide,str(ligne_1[index])) and appartenance_au_moins_un(mention_vide,str(ligne_2[index])):
                    correspondance.append(index+1)

            elif "nan" in str(ligne_1[index]) and "THI à J 180" in data.iloc[index1].values[0]:
                ligne_1[index]=data.iloc[255].values[index+1]
                if ligne_1[index] != ligne_2[index]:
                    if appartenance_au_moins_un(mention_vide,str(ligne_1[index])):
                        ligne_1[index]=data.iloc[251].values[index+1]
                        if ligne_1[index] == ligne_2[index]:
                            correspondance.append(index+1)  
                else:
                    correspondance.append((index+1))
        else:
            correspondance.append(index+1)
    print(Fore.CYAN+ subject + Style.RESET_ALL)
    for index in range(i):
            if (index+1) not in correspondance:
                print("############################")
                print(f"Erreur de correspondance : index {index+1}")
                print(f"Ligne_1[{index+1}] = {ligne_1[index]}")
                print(f"Ligne_2[{index+1}] = {ligne_2[index]}")
                non_correspondance.append(index+1)
#Creation tab des valeurs des patients dans correspondance
   # tab_id_patients_correspondance = creation_tab_colonne_personnalisé(correspondance)
    tab_valeurs_correspondantes={}
    for index in correspondance:
        tab_valeurs_correspondantes[index]=data.iloc[index1].values[index]
    #print("Poucentage d'erreur: ", (cnt/len(THI_1))*100,"%")
    print(correspondance)
    print(tab_valeurs_correspondantes)
    print(Fore.YELLOW+"Pourcentage correspondance :",len(correspondance)/len(ligne_1)*100,"%"+Style.RESET_ALL)
    return tab_valeurs_correspondantes, correspondance,non_correspondance

   

1 - Evaluation initiale et finale

1.1 - THI initial

- Récupération des index des lignes du THI initial

In [150]:
ligne_THI_initial = data[data['Unnamed: 0']== 'THI initial'].index[0]
print(ligne_THI_initial)

247


In [151]:
ligne_THI_initial_maj = data[data['Unnamed: 0']== 'THI INITIAL'].index[0]
print(ligne_THI_initial_maj)

335


- Cohérence du THI initial

In [152]:
tab_valeurs_correspondantes_THI_initial, correspondance_THI_initial, non_correpondance_THI_initial = correspondance_simple(ligne_THI_initial,ligne_THI_initial_maj, "Correspondance THI initial et THI initial mis à jour",taille)

Correspondance THI initial et THI initial mis à jour
############################
Erreur de correspondance : index 6
Ligne_1[6] = 86
Ligne_2[6] = 78
############################
Erreur de correspondance : index 7
Ligne_1[7] = 52
Ligne_2[7] = 46
############################
Erreur de correspondance : index 9
Ligne_1[9] = 52
Ligne_2[9] = 56
############################
Erreur de correspondance : index 10
Ligne_1[10] = 24
Ligne_2[10] = 28
############################
Erreur de correspondance : index 12
Ligne_1[12] = 78
Ligne_2[12] = 70
############################
Erreur de correspondance : index 56
Ligne_1[56] = 70
Ligne_2[56] = 44
############################
Erreur de correspondance : index 57
Ligne_1[57] = 44
Ligne_2[57] = 38
############################
Erreur de correspondance : index 58
Ligne_1[58] = 38
Ligne_2[58] = 54
############################
Erreur de correspondance : index 59
Ligne_1[59] = 54
Ligne_2[59] = 31
############################
Erreur de correspondance : index 60


Obs:

On remarque qu'il y a seulement une correspondance de 52 % entre les deux lignes de THI initial ce qui est peu.
- Avons nous bien compris la lecture des données?
- Est-ce que c'est un decallage dans les données ?

Nous allons vérifier cela très prochainement :

Si le pourcentage de l'un ou de l'autre est supérieur a 52 % on pourra alors envisager l'une de ces hypotheses.

1.1.1 - Decallage des données

In [153]:
THI_initial_2_patient_i = data.iloc[335].values[1:]
THI_initial_1_patient_i_after= data.iloc[247].values[1:]

i = 0
while pd.notna(THI_initial_2_patient_i[i]):
    i += 1
THI_initial_1_patient_i_after = THI_initial_1_patient_i_after[1:i+2]


cnt=0
for index in range(i+1):
    print("index",index)
    print(THI_initial_2_patient_i[index] ,"=", THI_initial_1_patient_i_after[index])
    if THI_initial_2_patient_i[index] == THI_initial_1_patient_i_after[index]:
        print("############################")
        print(f"Correspondance : patient {index}-{index+1}")
        print(f"THI_initial_2_patient_i[{index}] = {THI_initial_2_patient_i[index]}")
        print(f"THI_initial_1_patient_i_after[{index}] = {THI_initial_1_patient_i_after[index]}")
        cnt+=1

print("Poucentage de correspondance: ", (cnt/(len(THI_initial_1_patient_i_after)-1))*100,"%")

index 0
32 = 28
index 1
28 = 64
index 2
64 = 30
index 3
30 = 24
index 4
24 = 86
index 5
78 = 52
index 6
46 = 56
index 7
56 = 52
index 8
56 = 24
index 9
28 = 30
index 10
30 = 78
index 11
70 = 28
index 12
28 = 51
index 13
51 = 64
index 14
64 = 38
index 15
38 = 48
index 16
48 = 24
index 17
24 = 14
index 18
14 = 64
index 19
64 = 62
index 20
62 = 66
index 21
66 = 54
index 22
54 = 56
index 23
56 = 38
index 24
38 = 22
index 25
22 = 20
index 26
20 = 24
index 27
24 = 44
index 28
44 = 68
index 29
68 = 66
index 30
66 = 22
index 31
22 = 42
index 32
42 = 72
index 33
72 = 74
index 34
74 = 18
index 35
18 = 10
index 36
10 = 44
index 37
44 = 52
index 38
52 = 66
index 39
66 = 16
index 40
16 = 64
index 41
64 = 32
index 42
32 = 46
index 43
46 = 22
index 44
22 = 22
############################
Correspondance : patient 44-45
THI_initial_2_patient_i[44] = 22
THI_initial_1_patient_i_after[44] = 22
index 45
22 = 44
index 46
44 = 30
index 47
30 = 98
index 48
98 = 36
index 49
36 = 34
index 50
34 = 46
index 51
46

1.1.2 - HYPOTHESE 2 : Mauvaise compréhension des données

Apres observation de quelques lignes, on remarque que le THI initial des outputs(n°2) pourrait être celui à J60

- Récupération des lignes de THI initial J 60

In [154]:
ligne_THI_initital_J60 = data[data["Unnamed: 0"]=="THI à J 60"].index[0]
print(ligne_THI_initital_J60)


251


In [155]:
tab_valeurs_correspondantes_THI_initial_hypoJ60, correspondance_THI_initial_J60, non_correspondance_THI_initial_J60 = correspondance_simple(ligne_THI_initital_J60, ligne_THI_initial_maj,"Corespondance THI J60 et THI initial mis à jour",taille)

Corespondance THI J60 et THI initial mis à jour
############################
Erreur de correspondance : index 3
Ligne_1[3] = 58
Ligne_2[3] = 64
############################
Erreur de correspondance : index 15
Ligne_1[15] = 60
Ligne_2[15] = 64
############################
Erreur de correspondance : index 16
Ligne_1[16] = 18
Ligne_2[16] = 38
############################
Erreur de correspondance : index 17
Ligne_1[17] = 40
Ligne_2[17] = 48
############################
Erreur de correspondance : index 21
Ligne_1[21] = 48
Ligne_2[21] = 62
############################
Erreur de correspondance : index 23
Ligne_1[23] = 46
Ligne_2[23] = 54
############################
Erreur de correspondance : index 24
Ligne_1[24] = 30
Ligne_2[24] = 56
############################
Erreur de correspondance : index 25
Ligne_1[25] = 36
Ligne_2[25] = 38
############################
Erreur de correspondance : index 26
Ligne_1[26] = 16
Ligne_2[26] = 22
############################
Erreur de correspondance : index 27

Conclusion: Il n'y a pas de lien avec THI J60 en particulier

1.2 - THI final

- Récupération des lignes de THI J180 et THI final

In [156]:
ligne_THI_final_J180 = data[data["Unnamed: 0"] == "THI à J 180"].index[0]
print(ligne_THI_final_J180)

259


In [157]:
ligne_THI_final = data[data["Unnamed: 0"] == " THI FINAL"].index[0]
print(ligne_THI_final)

336


Verification Coherence entre le THI final du tableau update et le THI final dans les outputs

In [158]:
tab_valeurs_correspondantes_THI_final, correspondance_THI_final, non_correspondance_THI_final = correspondance_simple(ligne_THI_final_J180, ligne_THI_final," Correspondance THI final et THI final mis à jour",taille)

 Correspondance THI final et THI final mis à jour
############################
Erreur de correspondance : index 12
Ligne_1[12] = 62
Ligne_2[12] = 42
############################
Erreur de correspondance : index 15
Ligne_1[15] = 60
Ligne_2[15] = 64
############################
Erreur de correspondance : index 25
Ligne_1[25] = 36
Ligne_2[25] = 30
############################
Erreur de correspondance : index 40
Ligne_1[40] = 20
Ligne_2[40] = 32
############################
Erreur de correspondance : index 42
Ligne_1[42] = 16
Ligne_2[42] = 25
############################
Erreur de correspondance : index 55
Ligne_1[55] = 40
Ligne_2[55] = 20
############################
Erreur de correspondance : index 56
Ligne_1[56] = 20
Ligne_2[56] = 10
############################
Erreur de correspondance : index 57
Ligne_1[57] = 10
Ligne_2[57] = 6
############################
Erreur de correspondance : index 58
Ligne_1[58] = 6
Ligne_2[58] = 8
############################
Erreur de correspondance : index 

Obs:

- Probleme de coherence entre le THI initial de base et le nouveau: 52% de correspondance
- Concernant le THI final, il y a aussi un probleme de coherence : 55% de correspondance

2 -  Efficacité de la prise en charge

Voici une fonction qui va permettre de mettre en forme les decisions multiples d'un meme theme faites pour chaque patient

In [159]:
def creation_tableau_choix_multiple(lignes):
    tableau_general=[]
    for index in range(i):
        tableau_patient=[]
        for index1 in lignes:
            tableau_patient.append(data.iloc[index1].values[index+1])
        tableau_general.append(tableau_patient)
        
    print(tableau_general)
    print(len(tableau_general))
    return tableau_general

Voici une fonction qui va rescensser le nombre de + de chaque str

In [160]:
def count_plus(item):
    return item.count('+')

2.1 - Efficacité des médicament

- Récupération des lignes suivantes :

In [161]:
ligne_efficacite_vasculotropes = data[data["Unnamed: 0"]=="Efficacité des vasculotropes"].index[0]
ligne_efficacite_solupred = data[data["Unnamed: 0"]=="Eficacité du Solupred (prednisolone)"].index[0]
ligne_efficacite_neurontin = data[data["Unnamed: 0"]=="Efficacité du Neurontin (gabapentine)"].index[0]
ligne_efficacite_tégrétol = data[data["Unnamed: 0"]=="Efficacité du Tégrétol (carbamazépine)"].index[0]
ligne_efficacite_lamictal = data[data["Unnamed: 0"]=="Efficacité du Lamictal (lamotrigine)"].index[0]
ligne_efficacite_epitomax = data[data["Unnamed: 0"]=="Efficacité de l'Epitomax (topiramate)"].index[0]
ligne_efficacite_lyrica = data[data["Unnamed: 0"]=="Efficacité du Lyrica (pregabaline)"].index[0]
ligne_efficacite_depakine = data[data["Unnamed: 0"]=="Efficacité de la Dépakine (valproate de sodium)"].index[0]
ligne_efficacite_rivotril = data[data["Unnamed: 0"]=="Efficacité du Rivotril (clonazepam)"].index[0]

In [162]:
tab_efficacite_medicament_par_patient = creation_tableau_choix_multiple([ligne_efficacite_vasculotropes, ligne_efficacite_solupred, ligne_efficacite_neurontin, ligne_efficacite_tégrétol, ligne_efficacite_lamictal, ligne_efficacite_epitomax, ligne_efficacite_lyrica, ligne_efficacite_depakine, ligne_efficacite_rivotril])

[[nan, nan, 'N +++', nan, nan, nan, nan, nan, nan], [nan, nan, 'N ++', 'T ++', 'L ++', 'E ++', nan, nan, nan], [nan, nan, 'N +++', 'T ++', 'L ++', nan, nan, nan, nan], [nan, nan, 'N +++', nan, nan, nan, nan, nan, nan], [nan, nan, 'N +++', nan, 'L ++', nan, nan, nan, nan], [nan, nan, 'N +', 'T +++', nan, 'E +++', nan, nan, nan], [nan, nan, 'N +', 'T ++', nan, nan, nan, nan, nan], [nan, nan, 'N +', 'T +', nan, nan, nan, nan, nan], [nan, nan, 'N   0', 'T +', 'L ++', nan, nan, 'D +++', 'R +++'], [nan, nan, 'N ++', 'T +', nan, nan, nan, 'D +', nan], [nan, nan, 'N +++', nan, nan, nan, nan, nan, nan], [nan, nan, 'N +', 'T +++', nan, nan, nan, nan, nan], [nan, 'S +++', 'N ++', nan, nan, nan, nan, nan, nan], [nan, nan, 'N +', nan, nan, nan, nan, nan, nan], [nan, nan, 'N  0', 'T 0', nan, nan, nan, nan, nan], [nan, nan, 'N +++', 'T +++', 'L ++', 'E ++', nan, nan, nan], [nan, nan, 'N ++', 'T ++', 'L ++', nan, nan, 'D ++', nan], [nan, nan, 'N +++', 'T +++', nan, nan, nan, nan, nan], [nan, nan, 'N +

Voici un code qui va permettre de choisir pour chaque patient, le meilleur médicament en fonction du nombre de +: 
- Plus il y a de + plus le médicament a été efficace (vice versa)
- Lorsque le nombre de + est égal, alors on prendra le médicament le plus vers le haut

In [163]:
def meilleur_médicament() :
    tab1 = []
    tab_meilleur_medicament_par_patient = []
    
    #Supression des nan dans le tableau_efficacite_medicament_par_patient
    for index in tab_efficacite_medicament_par_patient:
        tab2 = []
        for item in index:
            if isinstance(item, str) and mention_vide[0] not in item:
                tab2.append(item)
        if tab2 or not any(isinstance(item, str) and mention_vide[0] not in item for item in index):
            tab1.append(tab2)
    
    #Choix du médicament avec le plus de + 
    #En cas d'égalité, choix du médicament le plus vers le haut
    for index2 in tab1:
        sous_liste_triee = sorted(index2, key=count_plus, reverse=True)
        if sous_liste_triee:  
            premier_element = sous_liste_triee[0]
            if mention_vide[1] in premier_element:
                premier_element = np.nan
            tab_meilleur_medicament_par_patient.append(premier_element)
        else:
            tab_meilleur_medicament_par_patient.append(np.nan)

    print(tab_meilleur_medicament_par_patient)
    print(len(tab_meilleur_medicament_par_patient))
    return tab_meilleur_medicament_par_patient


In [164]:
tab_meilleur_medicament_par_patient = meilleur_médicament()

['N +++', 'N ++', 'N +++', 'N +++', 'N +++', 'T +++', 'T ++', 'N +', 'D +++', 'N ++', 'N +++', 'T +++', 'S +++', 'N +', nan, 'N +++', 'N ++', 'N +++', 'N ++++', 'N +++', 'E +++', 'N +++', 'T ++', 'N ++', 'S ++', 'N +++', 'N +++', 'N +++', 'N ++', 'R +++', 'N ++', 'N +++', 'N +++', 'N +++', 'N +', 'N +++', 'N +++', 'N +', nan, 'N +++', 'N +++', 'N +++', 'N +++', 'N +++', 'N +++', 'N +++', 'N +', 'T +++', 'N +++', 'N +++', 'N +++', 'N +++', 'D +++', 'N ++', 'N +', 'T +++', 'N +++', 'N +++', 'E ++', 'N ++', 'N +++', 'N +++', 'N +++', 'N +++', 'N ++', 'N +++', 'N +++', 'N ++', 'S ++', 'V ++', 'N ++', 'N +++', 'N +++', 'N +++', 'NN +++', 'N +', 'N +++', 'N +', 'N +++', 'N ++', 'N ++', 'E ++', 'N +', 'N +', 'D ++', 'N ++', 'N +++', 'N +++', 'N +++', 'N +++', 'N ++', 'N +++', 'T ++', 'N +++', 'N +++', 'N +++', 'N +++', 'N +++', nan, 'T +']
100


Voici le code qui permet de deduire la combinaison (médicament, appareillage) par notre logique

In [165]:
def combinaison_medicament_appareillage():
    tab_assemblement_medicament_appareillage = []

    appareillage=data.iloc[321].values[1:]
    for index in range(len(tab_meilleur_medicament_par_patient)):
        if count_plus(str(tab_meilleur_medicament_par_patient[index]))== 0 and count_plus(str(appareillage[index])) == 0:
            tab_assemblement_medicament_appareillage.append([np.nan,np.nan])
        elif count_plus(str(tab_meilleur_medicament_par_patient[index])) == count_plus(str(appareillage[index])):
            tab_assemblement_medicament_appareillage.append([tab_meilleur_medicament_par_patient[index][0],"A"])
        elif count_plus(str(tab_meilleur_medicament_par_patient[index])) > count_plus(str(appareillage[index])):
            tab_assemblement_medicament_appareillage.append([tab_meilleur_medicament_par_patient[index][0],np.nan])
        elif  count_plus(str(tab_meilleur_medicament_par_patient[index])) < count_plus(str(appareillage[index])):
            tab_assemblement_medicament_appareillage.append([np.nan,"A"])
            
    print(tab_assemblement_medicament_appareillage)
    print(len(tab_assemblement_medicament_appareillage))
    return list(tab_assemblement_medicament_appareillage)


In [166]:
tab_assemblement_medicament_appareillage = combinaison_medicament_appareillage()
print(type(tab_assemblement_medicament_appareillage))

[['N', 'A'], [nan, 'A'], ['N', nan], ['N', 'A'], ['N', nan], ['T', nan], [nan, 'A'], [nan, 'A'], ['D', nan], [nan, 'A'], ['N', nan], ['T', nan], ['S', 'A'], ['N', nan], [nan, nan], ['N', nan], ['N', nan], ['N', 'A'], ['N', nan], ['N', 'A'], ['E', 'A'], ['N', 'A'], [nan, 'A'], ['N', nan], ['S', nan], ['N', nan], ['N', nan], ['N', nan], ['N', nan], ['R', 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], [nan, 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['T', 'A'], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', nan], ['D', 'A'], ['N', nan], ['N', nan], ['T', 'A'], ['N', nan], ['N', 'A'], [nan, 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], [nan, 'A'], [nan, 'A'], ['V', nan], ['N', nan], ['N', nan], ['N', nan], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['E', 'A'], ['N', nan], [na

Voici les données mises à jour du tableau pour la combinaison (médicament, appareillage)

In [167]:
tableau_general_medic_appareillage=creation_tableau_choix_multiple([332,333])

[['N', 'A'], [nan, 'A'], ['N', nan], ['N', 'A'], ['N', nan], ['T', nan], ['T', 'A'], [nan, 'A'], ['R', nan], [nan, 'A'], ['N', nan], ['T', nan], ['N', 'A'], ['0', '0'], [nan, nan], ['N', nan], ['N', nan], ['N', 'A'], ['N', nan], ['N', 'A'], ['E', 'A'], ['N', 'A'], [nan, 'A'], ['N', nan], [nan, nan], ['N / T', nan], ['N', nan], ['N', nan], ['N', nan], ['R', 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], [nan, 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', 'A'], ['N', 'A'], [nan, nan], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', nan], ['N', nan], [nan, 'A'], ['N', nan], [nan, nan], ['T', 'A'], ['N', nan], ['N', 'A'], [nan, 'A'], ['N', nan], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], ['N', 'A'], [nan, 'A'], ['N', nan], ['N', nan], ['N / T', nan], ['N', nan], ['N', nan], ['N', 'A'], [nan, 'A'], ['N', 'A'], ['N', nan], ['N', 'A'], ['N', 'A'], ['N', nan], [nan, 'A'], ['N', n

Voici la fonction qui permet de voir la correspondance entre tableaux multiples:

In [168]:
def correspondance_multiple(ligne,tab1,tab2,subject):
    non_correspondance=[]
    print(Fore.CYAN + subject + Style.RESET_ALL )
    for i in range(len(tab1)):
        if tab1[i] != tab2[i]:
            print("############################")
            print(f"Erreur de correspondance : index {i+1}")
            print(f"tableau_general1[{i+1}] = {tab1[i]}")
            print(f"tableau_general2[{i+1}] = {tab2[i]}")
            non_correspondance.append(i+1)
    print(non_correspondance)
            
    correspondance=[]
    for index in range(1,taille) :
        if index not in non_correspondance:
            correspondance.append(index)
    print(len(correspondance))
            
    tab_valeurs_correspondantes = {}
    for index0 in range(len(ligne)):
        sous_table = {}
        for index in correspondance:
            sous_table[index] = data.iloc[ligne[index0]].values[index]
        
        tab_valeurs_correspondantes[index0] = sous_table

    print(tab_valeurs_correspondantes)
    print(len(tab_valeurs_correspondantes))

        
    print(Fore.YELLOW +"Pourcentage de correspondance: " , (len(correspondance)/len(tab1))*100, "%" + Style.RESET_ALL)
    print(correspondance)
    return tab_valeurs_correspondantes, correspondance

In [169]:
tab_valeurs_correspondantes_medicament_appareillage ,correspondance_medicament_appareillage=correspondance_multiple([332,333],tab_assemblement_medicament_appareillage,tableau_general_medic_appareillage,"Correspondance entre la combinaison [médicament,appareil] trouvée et celle mise à jour dans le tableau")

Correspondance entre la combinaison [médicament,appareil] trouvée et celle mise à jour dans le tableau
############################
Erreur de correspondance : index 7
tableau_general1[7] = [nan, 'A']
tableau_general2[7] = ['T', 'A']
############################
Erreur de correspondance : index 9
tableau_general1[9] = ['D', nan]
tableau_general2[9] = ['R', nan]
############################
Erreur de correspondance : index 13
tableau_general1[13] = ['S', 'A']
tableau_general2[13] = ['N', 'A']
############################
Erreur de correspondance : index 14
tableau_general1[14] = ['N', nan]
tableau_general2[14] = ['0', '0']
############################
Erreur de correspondance : index 25
tableau_general1[25] = ['S', nan]
tableau_general2[25] = [nan, nan]
############################
Erreur de correspondance : index 26
tableau_general1[26] = ['N', nan]
tableau_general2[26] = ['N / T', nan]
############################
Erreur de correspondance : index 47
tableau_general1[47] = ['N', nan]
ta

2.2 - Efficacité de l'appareillage

- Récupération ligne : "Efficacité de l'appareillage auditif" et "Appareillage"

In [170]:
ligne_efficacite_appareillage_auditif = data[data["Unnamed: 0"]=="Efficacité de l'appareillage auditif"].index[0]
ligne_appareillage_maj = data[data["Unnamed: 0"]=="Appareillage"].index[0]

In [171]:
tab_valeurs_correspondantes_appareillage, correspondance_appareillage, non_correspondance_appareillage=correspondance_simple(ligne_efficacite_appareillage_auditif, ligne_appareillage_maj, "Appareillage",i)
print(non_correspondance_appareillage)

Appareillage
############################
Erreur de correspondance : index 17
Ligne_1[17] = APP +
Ligne_2[17] = nan
############################
Erreur de correspondance : index 81
Ligne_1[81] = APP +
Ligne_2[81] = nan
############################
Erreur de correspondance : index 99
Ligne_1[99] = APP  0
Ligne_2[99] = nan
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100]
{1: 'APP +++', 2: 'APP +++', 3: nan, 4: 'APP +++', 5: nan, 6: nan, 7: 'APP +++', 8: 'APP +++', 9: nan, 10: 'APP +++', 11: nan, 12: nan, 13: 'APP +++', 14: nan, 15: nan, 16: nan, 18: 'APP +++', 19: nan, 20: 'APP +++', 21: 'APP +++', 22: 'APP +++', 23: 'APP +++', 24: nan, 25: nan, 26: nan, 27: nan

On se rend compte ici qu'il y a des pb de coherence. Ainsi on a une theorie qui dit que lorsque APP a plus de + que le medicament ayant le plus + alors on le met sinon on le met pas ="nan"

- récupération ligne appareillage

In [172]:
print(ligne_efficacite_appareillage_auditif)

321


In [173]:
def choix_appareil_ou_non():
    while non_correspondance_appareillage:
        for a in non_correspondance_appareillage[:]:
            m = data.iloc[ligne_efficacite_appareillage_auditif].values[a]
            if count_plus(m) < count_plus(tab_general_trie[a]):
                data.iloc[ligne_efficacite_appareillage_auditif, a] = np.nan
                correspondance_appareillage.append(a)
                non_correspondance_appareillage.remove(a)  
                break  

In [174]:
print(correspondance_appareillage)
print(non_correspondance_appareillage)
print("Pourcantage correspondance :",len(correspondance_appareillage)/i*100,"%")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100]
[17, 81, 99]
Pourcantage correspondance : 97.0 %


3 - Protocole par patient

--> Données de base
- Traitement médical seul --> index = 231
- Appareillage auditif seul --> index = 232
- Appareillage auditif et traitement médical --> index = 233
- Traitement médical puis appareillage auditif --> index 234

--> Données mises à jour
- Traitement médical seul --> index = 327
- Appareillage auditif seul --> index = 328
- Appareillage auditif et traitement médical --> index = 329
- Traitement médical puis appareillage auditif --> index 330

3.1 -  Tableau données de bases

- Récupération des lignes suivantes : Traitement médical seul, Appareillage auditif seul, Appareillage auditif et traitement médical, Traitement médical puis appareillage auditif

In [175]:
ligne_traitement_medical_seul = data[data["Unnamed: 0"]=="Traitement médical seul"].index[0]
ligne_appareillage_auditif_seul = data[data["Unnamed: 0"]=="Appareillage auditif seul"].index[0]
ligne_appareillage_medicament = data[data["Unnamed: 0"]=="Appareillage auditif et traitement médical"].index[0]
ligne_medicament_puis_appareillage = data[data["Unnamed: 0"]=="Traitement médical puis appareillage auditif"].index[0]

In [176]:
tableau_protocole_par_patient=creation_tableau_choix_multiple([ligne_traitement_medical_seul,ligne_appareillage_medicament,ligne_medicament_puis_appareillage,ligne_appareillage_auditif_seul])


[[nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, 'X', 'X', nan], [nan, nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, nan, 'X'], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan

3.2 -  Tableau données mises à jour

Les colonnes n'étant pas dans le meme ordre que pour les données initiales, il faudra inverser les indexes pour obtenir les memes combinaisons

In [177]:
tableau_protocole_par_patient_maj=creation_tableau_choix_multiple([327,328,329,330])

[[nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, 'X', nan, nan], [nan, nan, 'X', nan], [nan, nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, nan, nan], [nan, nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, 'X', nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], [nan, nan, nan, 'X'], ['X', nan, nan, nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan, 'X', nan], ['X', nan, nan, nan], [nan, nan, 'X', nan], [nan, nan

Vérification de la cohérence entre tableaux multiple

In [178]:
tab_valeurs_correspondantes_portocoles, correspondance_protocole=correspondance_multiple([327,330,328,329],tableau_protocole_par_patient,tableau_protocole_par_patient_maj,"Correspondance Protocoles")

Correspondance Protocoles
############################
Erreur de correspondance : index 7
tableau_general1[7] = [nan, nan, 'X', nan]
tableau_general2[7] = [nan, 'X', nan, nan]
############################
Erreur de correspondance : index 8
tableau_general1[8] = [nan, 'X', 'X', nan]
tableau_general2[8] = [nan, nan, 'X', nan]
############################
Erreur de correspondance : index 16
tableau_general1[16] = [nan, nan, nan, nan]
tableau_general2[16] = ['X', nan, nan, nan]
############################
Erreur de correspondance : index 19
tableau_general1[19] = [nan, nan, 'X', nan]
tableau_general2[19] = ['X', nan, nan, nan]
############################
Erreur de correspondance : index 20
tableau_general1[20] = [nan, nan, 'X', nan]
tableau_general2[20] = [nan, 'X', nan, nan]
############################
Erreur de correspondance : index 29
tableau_general1[29] = [nan, nan, 'X', nan]
tableau_general2[29] = ['X', nan, nan, nan]
############################
Erreur de correspondance : index 

II- Intersection des données

Le but ici est de définir un echantillons de patient dont les données sont fiables.

En résumé on a :

In [179]:
print("Correspondance médicament/appareillage")
print(correspondance_medicament_appareillage)
print(len(correspondance_medicament_appareillage))

print("Correspondance protocole")
print(correspondance_protocole)
print(len(correspondance_protocole))

print("Correspondance THI initial")
print(correspondance_THI_initial)
print(len(correspondance_THI_initial))

print("Correspondance THI final")
print(correspondance_THI_final)
print(len(correspondance_THI_final))



Correspondance médicament/appareillage
[1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50, 51, 52, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 71, 73, 74, 77, 78, 79, 80, 81, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
80
Correspondance protocole
[1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 71, 73, 75, 76, 77, 78, 79, 80, 81, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]
80
Correspondance THI initial
[1, 2, 3, 4, 5, 8, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 72, 86]
52
Correspondance THI final
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 

In [180]:
#intersection des indexs
nouvel_echantillon = set(correspondance_medicament_appareillage) & set(correspondance_THI_final) & set(correspondance_protocole) & set(correspondance_THI_initial)
print(list(nouvel_echantillon))
print(len(nouvel_echantillon))


[1, 2, 3, 4, 5, 11, 17, 18, 21, 22, 23, 24, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 41, 43, 44, 45, 46, 49, 50, 52, 54]
32


Conclusion : On a un nouvel echantillon de 32 patients

On doi donc mettre à jour les dictionnaires réalisés avec les fonctions précédentes. Ces dictionnaires ne doivent contenir que les valeurs associées aux indexs présents dans le nouvel echantillon

- Fonction pour les comparaisons simples

In [181]:
def maj_dicco_valeurs_correspondantes_simple(nouvel_echantillon, dicco_valeurs_correspondantes):
    common_dict = {}
    for key in nouvel_echantillon:
        if key in dicco_valeurs_correspondantes:
            common_dict[key] = dicco_valeurs_correspondantes[key]
    print(common_dict)
    print(len(common_dict))
    return common_dict


- Fonction pour les comparaisons multiples

In [182]:

def maj_dicco_valeurs_correspondantes_mult(nouvel_echantillon, dicco_valeurs_correspondantes):
    nouveau_dictionnaire = {}
    for cle, sous_dictionnaire in dicco_valeurs_correspondantes.items():
        sous_dictionnaire_filtre = {}
        for sous_cle, valeur in sous_dictionnaire.items():
            if sous_cle in nouvel_echantillon:
                sous_dictionnaire_filtre[sous_cle] = valeur
        nouveau_dictionnaire[cle] = sous_dictionnaire_filtre
        
    print(nouveau_dictionnaire)
    print(len(nouveau_dictionnaire[0]))
    return nouveau_dictionnaire

In [183]:
dicco_valeurs_correspondantes_THI_initial_maj=maj_dicco_valeurs_correspondantes_simple(nouvel_echantillon, tab_valeurs_correspondantes_THI_initial)

{1: '32', 2: '28', 3: '64', 4: '30', 5: '24', 11: '30', 17: '48', 18: '24', 21: '62', 22: '66', 23: '54', 24: '56', 27: '20', 28: '24', 30: '68', 31: '66', 32: '22', 33: '42', 34: '72', 36: '18', 37: '10', 38: '44', 39: '52', 41: '16', 43: '32', 44: '46', 45: '22', 46: '22', 49: '98', 50: '36', 52: '46', 54: '36'}
32


In [184]:
dicco_valeurs_correspondantes_THI_final_maj=maj_dicco_valeurs_correspondantes_simple(nouvel_echantillon, tab_valeurs_correspondantes_THI_final)

{1: '8', 2: '10', 3: '34', 4: '8', 5: '8', 11: '4', 17: '14', 18: '4', 21: '10', 22: '10', 23: '10', 24: '13', 27: '0', 28: '12', 30: '18', 31: '38', 32: '0', 33: '10', 34: '18', 36: '4', 37: '4', 38: '38', 39: '18', 41: '0', 43: '2', 44: '8', 45: '10', 46: '4', 49: '16', 50: '10', 52: '26', 54: '20'}
32


In [185]:
dicco_valeurs_correspondantes_medicament_appareille_maj = maj_dicco_valeurs_correspondantes_mult(nouvel_echantillon, tab_valeurs_correspondantes_medicament_appareillage)

{0: {1: 'N', 2: nan, 3: 'N', 4: 'N', 5: 'N', 11: 'N', 17: 'N', 18: 'N', 21: 'E', 22: 'N', 23: nan, 24: 'N', 27: 'N', 28: 'N', 30: 'R', 31: 'N', 32: 'N', 33: 'N', 34: 'N', 36: 'N', 37: 'N', 38: 'N', 39: nan, 41: 'N', 43: 'N', 44: 'N', 45: 'N', 46: 'N', 49: 'N', 50: 'N', 52: 'N', 54: 'N'}, 1: {1: 'A', 2: 'A', 3: nan, 4: 'A', 5: nan, 11: nan, 17: nan, 18: 'A', 21: 'A', 22: 'A', 23: 'A', 24: nan, 27: nan, 28: nan, 30: 'A', 31: nan, 32: nan, 33: 'A', 34: 'A', 36: nan, 37: 'A', 38: 'A', 39: 'A', 41: nan, 43: 'A', 44: nan, 45: 'A', 46: 'A', 49: 'A', 50: 'A', 52: nan, 54: nan}}
32


In [186]:
dicco_valeurs_correspondantes_protocole_maj= maj_dicco_valeurs_correspondantes_mult(nouvel_echantillon, tab_valeurs_correspondantes_portocoles)

{0: {1: nan, 2: nan, 3: 'X', 4: nan, 5: 'X', 11: nan, 17: nan, 18: nan, 21: nan, 22: nan, 23: nan, 24: 'X', 27: 'X', 28: 'X', 30: nan, 31: 'X', 32: 'X', 33: nan, 34: nan, 36: 'X', 37: nan, 38: nan, 39: nan, 41: 'X', 43: nan, 44: 'X', 45: nan, 46: nan, 49: nan, 50: nan, 52: 'X', 54: 'X'}, 1: {1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 11: nan, 17: nan, 18: nan, 21: nan, 22: nan, 23: nan, 24: nan, 27: nan, 28: nan, 30: nan, 31: nan, 32: nan, 33: nan, 34: nan, 36: nan, 37: nan, 38: nan, 39: 'X', 41: nan, 43: nan, 44: nan, 45: nan, 46: nan, 49: nan, 50: nan, 52: nan, 54: nan}, 2: {1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 11: nan, 17: nan, 18: nan, 21: nan, 22: nan, 23: nan, 24: nan, 27: nan, 28: nan, 30: nan, 31: nan, 32: nan, 33: nan, 34: nan, 36: nan, 37: nan, 38: nan, 39: nan, 41: nan, 43: nan, 44: nan, 45: nan, 46: nan, 49: nan, 50: nan, 52: nan, 54: nan}, 3: {1: 'X', 2: 'X', 3: nan, 4: 'X', 5: nan, 11: nan, 17: 'X', 18: 'X', 21: 'X', 22: 'X', 23: 'X', 24: nan, 27: nan, 28: nan, 30: 'X', 3

- Fonction pour attribuer à plusieurs variables les sous-dicco des dicco a correspondance mult

In [187]:
def separation_dicco(dicco_valeurs_mult):
    sous_diccos = []
    for valeur in dicco_valeurs_mult.values():
        if isinstance(valeur, dict):
            sous_diccos.append(valeur)
    return tuple(sous_diccos)

III. Algorithme de conclusion

In [188]:
def algo_deduction(taille, nouvel_echantillon):
    dicco_THI_final_maj={}
    tab_valeurs_correspondantes_THI_final, correspondance_THI_final, non_correspondance_THI_final = correspondance_simple(ligne_THI_final_J180, ligne_THI_final," Correspondance THI final et THI final mis à jour",taille)

    for k in nouvel_echantillon : 
        dicco_THI_final_maj[k]= tab_valeurs_correspondantes_THI_final[k]
    print(dicco_THI_final_maj)
    dicco_algo_conclusion={}
    for v in dicco_THI_final_maj:
        if  int(dicco_THI_final_maj[v])<=10:
            dicco_algo_conclusion[v]="Très Ben"
        elif int(dicco_THI_final_maj[v])<=20:
            dicco_algo_conclusion[v]="Bien"
        elif int(dicco_THI_final_maj[v])>20:
            dicco_algo_conclusion[v]="Echec"
        
    print(dicco_algo_conclusion)
    print(len(dicco_algo_conclusion))


In [189]:
algo_deduction(taille,nouvel_echantillon)

 Correspondance THI final et THI final mis à jour
############################
Erreur de correspondance : index 12
Ligne_1[12] = 62
Ligne_2[12] = 42
############################
Erreur de correspondance : index 15
Ligne_1[15] = 60
Ligne_2[15] = 64
############################
Erreur de correspondance : index 25
Ligne_1[25] = 36
Ligne_2[25] = 30
############################
Erreur de correspondance : index 40
Ligne_1[40] = 20
Ligne_2[40] = 32
############################
Erreur de correspondance : index 42
Ligne_1[42] = 16
Ligne_2[42] = 25
############################
Erreur de correspondance : index 55
Ligne_1[55] = 40
Ligne_2[55] = 20
############################
Erreur de correspondance : index 56
Ligne_1[56] = 20
Ligne_2[56] = 10
############################
Erreur de correspondance : index 57
Ligne_1[57] = 10
Ligne_2[57] = 6
############################
Erreur de correspondance : index 58
Ligne_1[58] = 6
Ligne_2[58] = 8
############################
Erreur de correspondance : index 

IV- Algorithme final de mise en forme

Cette algorithme va permettre de mettre en forme l'echantillon des 32 patients sous forme de data frame

- Voici les colonnes qui seront présentes dans le nouveau data frame

In [190]:
columns =[
    "patient_name",
    "Protocole",
    "Médicament",
    "Appareillage",
    "THI INITIAL",
    "THI FINAL" 
]

In [191]:
def creation_tab_colonne_personnalisé(liste):
    tab={}
    for i in nouvel_echantillon:
        tab[i]=data.columns[i]
    return tab

In [192]:

dicco_id_patients=creation_tab_colonne_personnalisé(nouvel_echantillon)
print(dicco_id_patients)


{1: '1/311853', 2: '2/338686', 3: '3/341194', 4: '4/4/303933', 5: '5/231942', 11: '11/296771', 17: '17/336174', 18: '18/293014', 21: '21/345335', 22: '22/211972', 23: '23/345137', 24: '24/145843', 27: '27:346266', 28: '28/87191', 30: '30/226000', 31: '31/101196', 32: '32/001706', 33: '33/298904', 34: '34/348945', 36: '36/350555', 37: '37/350555', 38: '38/104963', 39: '39/072487', 41: '41/027272', 43: '43/080809', 44: '44/019958', 45: '45/312821', 46: '46/230247', 49: '49/165468', 50: '50/353348', 52: '52/288249', 54: '54/237697'}


In [193]:
# sortie_finales_sans_evaluation=data.iloc[326:337,0]
# data.loc[326:337, tab_id_patients]

def algorithme_final(ligne_THI_initial , ligne_THI_initial_maj , ligne_THI_final , ligne_THI_final_maj,tableau_protocole_par_patient,tableau_protocole_par_patient_maj,tab_assemblement_medicament_appareillage,tableau_general_medic_appareillage,ligne1,ligne2):
    
    print(Back.LIGHTYELLOW_EX+ Fore.BLACK +"Voici le tableau des erreurs"+ Style.RESET_ALL)
    
    #Les fonctions de correspondance
    tab_valeurs_correspondantes_THI_initial,correspondance_THI_initial, non_correpondance_THI_initial = correspondance_simple(ligne_THI_initial,ligne_THI_initial_maj,"Etude de la cohérence du THI initial",i)
    tab_valeurs_correspondantes_THI_final,correspondance_THI_final,non_correspondance_THI_final = correspondance_simple(ligne_THI_final,ligne_THI_final_maj,"Etude de la cohérence du THI final",i)
    tab_valeurs_correspondantes_medicament_appareillage,correspondance_medicament_appareillage=correspondance_multiple(ligne1, tab_assemblement_medicament_appareillage,tableau_general_medic_appareillage,"Correspondance entre la combinaison [médicament,appareil] trouvée et celle mise à jour dans le tableau")
    tab_valeurs_correspondantes_portocoles, correspondance_protocole = correspondance_multiple(ligne2, tableau_protocole_par_patient,tableau_protocole_par_patient_maj,"Correspondance Protocoles")
    
    #Creation du nouvel echantillon
    nouvel_echantillon = set(correspondance_medicament_appareillage) & set(correspondance_THI_final) & set(correspondance_protocole) & set(correspondance_THI_initial)
    
    #Adptation des dicco crées dans les fonctions précédentes avec uniquement les index (en keys) du nouvel echantillon
    dicco_valeurs_correspondantes_THI_initial_maj=maj_dicco_valeurs_correspondantes_simple(nouvel_echantillon, tab_valeurs_correspondantes_THI_initial)
    dicco_valeurs_correspondantes_THI_final_maj=maj_dicco_valeurs_correspondantes_simple(nouvel_echantillon, tab_valeurs_correspondantes_THI_final)
    dicco_valeurs_correspondantes_medicament_appareil_maj = maj_dicco_valeurs_correspondantes_mult(nouvel_echantillon, tab_valeurs_correspondantes_medicament_appareillage)
    dicco_valeurs_correspondantes_protocole_maj= maj_dicco_valeurs_correspondantes_mult(nouvel_echantillon, tab_valeurs_correspondantes_portocoles)
    dicco_id_patients=creation_tab_colonne_personnalisé(nouvel_echantillon)
    
    #Separation des sous dicco des grand dicco a correspondance mult en plusieurs variables
    dicco_valeurs_correspondantes_traitement_med_seul,dicco_valeurs_correspondantes_appareil_et_med,dicco_valeurs_correspondantes_med_puis_appareil,dicco_valeurs_correspondantes_appareil_seul= separation_dicco(dicco_valeurs_correspondantes_protocole_maj)
    dicco_valeurs_correspondantes_appareillage, dicco_valeurs_correspondantes_medicament = separation_dicco(dicco_valeurs_correspondantes_medicament_appareil_maj)
    
    #creation du dicco protocole synthétisé
    dicco_protocole={}
    for k in  dicco_valeurs_correspondantes_traitement_med_seul :
        if "X" in str(dicco_valeurs_correspondantes_traitement_med_seul[k]) :
            dicco_protocole[k]= data.iloc[327].values[0]
        elif "X" in str(dicco_valeurs_correspondantes_appareil_et_med[k]) : 
            dicco_protocole[k]= data.iloc[328].values[0]
        elif "X" in str(dicco_valeurs_correspondantes_med_puis_appareil[k]) : 
            dicco_protocole[k]= data.iloc[329].values[0]
        elif "X" in str(dicco_valeurs_correspondantes_appareil_seul[k]) : 
            dicco_protocole[k]= data.iloc[330].values[0]
    print("dicco protocole", dicco_protocole)
    
    #Creation du pat_dict
    dicc=[dicco_id_patients, dicco_protocole, dicco_valeurs_correspondantes_appareillage,dicco_valeurs_correspondantes_medicament,dicco_valeurs_correspondantes_THI_initial_maj,dicco_valeurs_correspondantes_THI_final_maj]
    big_dicco ={}
    for h in range(len(columns)):
        big_dicco[columns[h]] = dicc[h]
    print(big_dicco)
   #Creation du dataFrame
    X_data_2 = pd.DataFrame(big_dicco).sort_index()
    display(X_data_2)    
    # tab_id_patients= creation_tab_colonne_personnalisé(nouvel_echantillon)
    # print(data.loc[326:337, tab_id_patients])
    
    


In [194]:
algorithme_final(ligne_THI_initial,ligne_THI_initial_maj, ligne_THI_final_J180, ligne_THI_final,tableau_protocole_par_patient,tableau_protocole_par_patient_maj,tab_assemblement_medicament_appareillage,tableau_general_medic_appareillage,[332,333],[ligne_traitement_medical_seul,ligne_appareillage_medicament,ligne_medicament_puis_appareillage,ligne_appareillage_auditif_seul])

Voici le tableau des erreurs
Etude de la cohérence du THI initial
############################
Erreur de correspondance : index 6
Ligne_1[6] = 86
Ligne_2[6] = 78
############################
Erreur de correspondance : index 7
Ligne_1[7] = 52
Ligne_2[7] = 46
############################
Erreur de correspondance : index 9
Ligne_1[9] = 52
Ligne_2[9] = 56
############################
Erreur de correspondance : index 10
Ligne_1[10] = 24
Ligne_2[10] = 28
############################
Erreur de correspondance : index 12
Ligne_1[12] = 78
Ligne_2[12] = 70
############################
Erreur de correspondance : index 56
Ligne_1[56] = 70
Ligne_2[56] = 44
############################
Erreur de correspondance : index 57
Ligne_1[57] = 44
Ligne_2[57] = 38
############################
Erreur de correspondance : index 58
Ligne_1[58] = 38
Ligne_2[58] = 54
############################
Erreur de correspondance : index 59
Ligne_1[59] = 54
Ligne_2[59] = 31
############################
Erreur de correspondanc

,patient_name,Protocole,Médicament,Appareillage,THI INITIAL,THI FINAL
1,1/311853,Traitement médical puis Appareillage,N,A,32,8
2,2/338686,Traitement médical puis Appareillage,NaN,A,28,10
3,3/341194,Traitement médical seul,N,NaN,64,34
4,4/4/303933,Traitement médical puis Appareillage,N,A,30,8
5,5/231942,Traitement médical seul,N,NaN,24,8
11,11/296771,NaN,N,NaN,30,4
17,17/336174,Traitement médical puis Appareillage,N,NaN,48,14
18,18/293014,Traitement médical puis Appareillage,N,A,24,4
21,21/345335,Traitement médical puis Appareillage,E,A,62,10
22,22/211972,Traitement médical puis Appareillage,N,A,66,10
